<a href="https://colab.research.google.com/github/loissnijders/Process-mining/blob/main/arima_lois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
from google.colab import drive

In [138]:
import io
import pandas as pd
import re
from datetime import timedelta
import numpy as np

In [139]:
!wget -O training.csv https://surfdrive.surf.nl/files/index.php/s/aNVxBfvZl273kKN/download   
!wget -O test.csv https://surfdrive.surf.nl/files/index.php/s/0lJ3j0RlMeychGj/download   


path_test = '/content/test.csv'
path_train = '/content/training.csv'

--2022-03-15 10:17:19--  https://surfdrive.surf.nl/files/index.php/s/aNVxBfvZl273kKN/download
Resolving surfdrive.surf.nl (surfdrive.surf.nl)... 145.100.27.67, 2001:610:108:203b:0:a11:da7a:5afe
Connecting to surfdrive.surf.nl (surfdrive.surf.nl)|145.100.27.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26049572 (25M) [text/csv]
Saving to: ‘training.csv’

training.csv        100%[===================>]  24.84M  16.8MB/s    in 1.5s    

2022-03-15 10:17:21 (16.8 MB/s) - ‘training.csv’ saved [26049572/26049572]

--2022-03-15 10:17:21--  https://surfdrive.surf.nl/files/index.php/s/0lJ3j0RlMeychGj/download
Resolving surfdrive.surf.nl (surfdrive.surf.nl)... 145.100.27.67, 2001:610:108:203b:0:a11:da7a:5afe
Connecting to surfdrive.surf.nl (surfdrive.surf.nl)|145.100.27.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5819692 (5.5M) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>]   5.55M  7.54MB/s    in 0.

In [140]:
path_test = '/content/test.csv'
path_train = '/content/training.csv'

In [141]:
# # This is for when you read the data in jupyther notebooks
# path_test = '/content/BPI_Challenge_2012-test.csv'
# path_train = '/content/BPI_Challenge_2012-training.csv'

In [142]:
df_train = pd.read_csv (path_train)
df_test = pd.read_csv (path_test)

In [143]:
# Keeping in test set only cases that started in test and not in train

train_cases = list(set(list(df_train['case concept:name'])))
test_cases = list(set(list(df_test['case concept:name'])))
# test_delete = [item for item in test_cases if item in train_cases]
test_keep = [item for item in test_cases if item not in train_cases]

df_test = df_test[df_test['case concept:name'].isin(test_keep)]

In [144]:
import matplotlib.pyplot as plt
import datetime

In [145]:
df_test['event time:timestamp']= pd.to_datetime(df_test['event time:timestamp'], dayfirst=True)
df_train['event time:timestamp']= pd.to_datetime(df_train['event time:timestamp'], dayfirst=True)

In [146]:
df_train = df_train.sort_values(by=['case concept:name', 'event time:timestamp'])

df_train = df_train.assign(nexttime=(df_train.groupby('case concept:name')['event time:timestamp'].shift(-1)))

In [147]:
df_test = df_test.sort_values(by=['case concept:name', 'event time:timestamp'])

df_test = df_test.assign(nexttime=(df_test.groupby('case concept:name')['event time:timestamp'].shift(-1)))

In [148]:
df_train['tonext'] = (df_train.nexttime - df_train['event time:timestamp']).astype('timedelta64[s]')
df_test['tonext'] = (df_test.nexttime - df_test['event time:timestamp']).astype('timedelta64[s]')

In [149]:
df_test = df_test.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
df_test.actionorder = df_test.actionorder + 1

In [150]:
df_train = df_train.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
df_train.actionorder = df_train.actionorder + 1

In [151]:
# targets = np.array(list(set(df_train['event concept:name'])))
# labelEnc = preprocessing.LabelEncoder()
# new_target = labelEnc.fit_transform(targets)
# onehotEnc = preprocessing.OneHotEncoder()
# onehotEnc.fit(new_target.reshape(-1, 1))
# targets_trans = onehotEnc.transform(new_target.reshape(-1, 1))
# print("The original data")
# print(targets)
# print("The transform data using OneHotEncoder")
# print(targets_trans.toarray())

In [152]:
# zip_iterator = zip(targets.tolist(), targets_trans.toarray().tolist())
# activity_dict = dict(zip_iterator)

# df_train['onehot'] = df_train['event concept:name']

In [153]:
# df_train['onehot'].replace(activity_dict, inplace=True)

In [154]:
import sklearn.preprocessing as preprocessing
import numpy as np
import pandas as pd

In [155]:
xtrain = df_train['event concept:name']
xtrain = xtrain.to_numpy()
xtest = df_test['event concept:name']
xtest = xtest.to_numpy()
x = np.concatenate((xtrain, xtest))

In [156]:
df_train['tonext']

0             0.0
1            53.0
2             0.0
3         39427.0
4           356.0
           ...   
214372        3.0
214373        NaN
214374        0.0
214375       39.0
214376        NaN
Name: tonext, Length: 214377, dtype: float64

In [157]:
xtrain2 = df_train.actionorder.to_numpy().reshape(-1, 1)
xtest2 = df_test.actionorder.to_numpy().reshape(-1, 1)

In [158]:
targets = x
labelEnc = preprocessing.LabelEncoder()
new_target = labelEnc.fit_transform(targets)
onehotEnc = preprocessing.OneHotEncoder()
onehotEnc.fit(new_target.reshape(-1, 1))
targets_trans = onehotEnc.transform(new_target.reshape(-1, 1))
print("The original data")
print(targets)
print("The transform data using OneHotEncoder")
print(targets_trans.toarray())

The original data
['A_SUBMITTED' 'A_PARTLYSUBMITTED' 'A_PREACCEPTED' ...
 'W_Afhandelen leads' 'A_DECLINED' 'W_Afhandelen leads']
The transform data using OneHotEncoder
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [159]:
x_train = targets_trans.toarray()[:214377]
x_test = targets_trans.toarray()[214377:]

In [160]:
x_train = np.concatenate((x_train, xtrain2), axis = 1)
x_test = np.concatenate((x_test, xtest2), axis = 1)

In [161]:
y_train = df_train.tonext.fillna(0).to_numpy()
y_test = df_test.tonext.fillna(0).to_numpy()

In [162]:
y_train

array([ 0., 53.,  0., ...,  0., 39.,  0.])

In [163]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((214377, 25), (214377,), (47823, 25), (47823,))

In [164]:
x_train, y_train

(array([[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 2.],
        [0., 0., 0., ..., 0., 0., 3.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 2.],
        [0., 0., 0., ..., 0., 0., 3.]]),
 array([ 0., 53.,  0., ...,  0., 39.,  0.]))

In [165]:
y_train

array([ 0., 53.,  0., ...,  0., 39.,  0.])

In [166]:
# y_train = y_train.reshape((214377, 1))
# y_test = y_test.reshape((47823, 1))

The time is now in seconds. Difficult to interpret so scale the seconds such that they are a value between 0 and 1. Where 1 is the longest time until a next activity. 

In [167]:
from sklearn.preprocessing import MinMaxScaler

In [168]:

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(np.concatenate((y_train, y_test)).reshape(-1, 1))

MinMaxScaler()

In [169]:
y_test.reshape(-1, 1)

array([[ 0.],
       [31.],
       [ 0.],
       ...,
       [50.],
       [ 4.],
       [ 0.]])

In [170]:
y_test = scaler.transform(y_test.reshape(-1, 1))
y_train = scaler.transform(y_train.reshape(-1, 1))

In [171]:
y_train

array([[0.00000000e+00],
       [5.96426125e-06],
       [0.00000000e+00],
       ...,
       [0.00000000e+00],
       [4.38879601e-06],
       [0.00000000e+00]])

In [172]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((214377, 25), (214377, 1), (47823, 25), (47823, 1))

In [173]:
y_test

array([[0.00000000e+00],
       [3.48853016e-06],
       [0.00000000e+00],
       ...,
       [5.62666155e-06],
       [4.50132924e-07],
       [0.00000000e+00]])

In [174]:
scaler.inverse_transform(y_test)

array([[ 0.],
       [31.],
       [ 0.],
       ...,
       [50.],
       [ 4.],
       [ 0.]])